<h2 align = center> Webscrapping </h2>

In order to obtain the KPIs needed for the project, the demographic information of the Ciudad Autónoma de Buenos Aires is added.

#### The libreries are imported

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### Extraction

The URL from which the information is taken is defined.

In [2]:
url = 'https://es.wikipedia.org/wiki/Buenos_Aires'

In [3]:
response = requests.get(url)
#content = response.txt

In [4]:
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-typography-survey-disabled vector-toc-available" dir="ltr" lang="es">
 <head>
  <meta charset="utf-8"/>
  <title>
   Buenos Aires - Wikipedia, la enciclopedia libre
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disa

In [6]:
# Se inicializa una variable llamada target_table como None
target_table = None

# Se buscan todas las etiquetas de tabla (<table>) en el contenido HTML
tables = soup.find_all('table')

# Se itera a través de todas las tablas encontradas en la página web
for table in tables:
    # Se busca una etiqueta de encabezado (<th>) dentro de la tabla que tenga los atributos colspan='3' y class='navbox-title'
    header = table.find('th', {'colspan': '3', 'class': 'navbox-title'})
    
    # Se comprueba si se encontró un encabezado y si el texto del encabezado contiene "Población histórica"
    if header and "Población histórica" in header.text:
        # Si se cumple la condición anterior, asigna la tabla actual a la variable target_table y sale del bucle for con break
        target_table = table
        break

In [7]:
target_table

<table class="toccolours" style="width:15em;border-spacing: 0;float:right;clear:right;margin:0 0 1em 1em;"><tbody><tr><th class="navbox-title" colspan="3" style="padding:0.25em;font-size:110%">Población histórica</th></tr><tr style="font-size:95%"><th style="border-bottom:1px solid black;padding:1px;width:3em">Año</th><th style="border-bottom:1px solid black;padding:1px 2px;text-align:right"><abbr title="Población">Pob.</abbr></th><th style="border-bottom:1px solid black;padding:1px;text-align:right"><abbr title="Cambio porcentual">±%</abbr></th></tr><tr><th style="text-align:center;padding:1px">1779 </th><td style="text-align:right;padding:1px">24 205</td><td style="text-align:right;padding:1px">—    </td></tr><tr><th style="text-align:center;padding:1px">1810 </th><td style="text-align:right;padding:1px">44 800</td><td style="text-align:right;padding:1px">+85.1%</td></tr><tr><th style="text-align:center;padding:1px">1869 </th><td style="text-align:right;padding:1px">177 797</td><td s

In [8]:
# Se comprueba si se encontró la tabla
if target_table:
    # Se crean listas para almacenar las columnas de la tabla
    years = []  
    populations = []  
    changes = [] 
    
    # Se inicializa una variable para almacenar el año actual
    current_year = None
    
    # Se itera a través de las filas de la tabla
    for row in target_table.find_all('tr'):
        # Se busca tanto en th (encabezados) como en td (celdas de datos)
        columns = row.find_all(['th', 'td'])  
        
        # Se comprueba si la fila tiene tres columnas (Año, Población y Cambio porcentual)
        if len(columns) == 3:
            current_year = columns[0].get_text(strip=True)  # Obtiene el año
            population = columns[1].get_text(strip=True)  # Obtiene la población
            change = columns[2].get_text(strip=True)  # Obtiene el cambio porcentual
            years.append(current_year)  # Agrega el año a la lista years
            populations.append(population)  # Agrega la población a la lista populations
            changes.append(change)  # Agrega el cambio porcentual a la lista changes
        elif len(columns) == 2 and current_year:
            # Si la fila tiene dos columnas y ya tenemos el año actual, 
            # se asume que es la población y el cambio porcentual
            population = columns[0].get_text(strip=True)  # Obtiene la población
            change = columns[1].get_text(strip=True)  # Obtiene el cambio porcentual
            populations.append(population)  # Agrega la población a la lista populations
            changes.append(change)  # Agrega el cambio porcentual a la lista changes
    
    # Se crea un DataFrame de Pandas con los datos extraídos
    df = pd.DataFrame({'Año': years, 'Población': populations, 'Cambio porcentual': changes})
    
    print("Ya se guardó el dataframe")
else:
    print("No se encontró la tabla deseada en la página.")

Ya se guardó el dataframe


In [9]:
df

,Año,Población,Cambio porcentual
0,Año,Pob.,±%
1,1779,24 205,—
2,1810,44 800,+85.1%
3,1869,177 797,+296.9%
4,1895,663 854,+273.4%
5,1914,1 575 814,+137.4%
6,1947,2 981 043,+89.2%
7,1960,2 966 634,−0.5%
8,1970,2 972 453,+0.2%
9,1980,2 922 829,−1.7%


The 'cambio porcentual' column is also removed as it is not needed for our analysis.

In [10]:
df.drop(df.index[0], inplace=True)
df

,Año,Población,Cambio porcentual
1,1779,24 205,—
2,1810,44 800,+85.1%
3,1869,177 797,+296.9%
4,1895,663 854,+273.4%
5,1914,1 575 814,+137.4%
6,1947,2 981 043,+89.2%
7,1960,2 966 634,−0.5%
8,1970,2 972 453,+0.2%
9,1980,2 922 829,−1.7%
10,1991,2 965 403,+1.5%


The first row is deleted as it is not a record.

In [11]:
del df['Cambio porcentual']
df

,Año,Población
1,1779,24 205
2,1810,44 800
3,1869,177 797
4,1895,663 854
5,1914,1 575 814
6,1947,2 981 043
7,1960,2 966 634
8,1970,2 972 453
9,1980,2 922 829
10,1991,2 965 403


The data type of the columns is checked.

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 1 to 13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Año        13 non-null     object
 1   Población  13 non-null     object
dtypes: object(2)
memory usage: 340.0+ bytes


In [18]:
df['Población'] = df['Población'].str.replace('\xa0', ' ')
df['Población'] = df['Población'].str.replace(' ', '')
df['Población'] = df['Población'].astype('float')
df['Año'] = df['Año'].astype('int')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 1 to 13
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Año        13 non-null     int32  
 1   Población  13 non-null     float64
dtypes: float64(1), int32(1)
memory usage: 288.0 bytes


And load data into a new csv file

In [22]:
df.to_csv('data/df_poblacion.csv', index = False, encoding='utf-8')

This is the end of this ETL, please click [here](01b_ETL_items.ipynb) to continue.